## This is to attempt a > 99.7% MNIST accuracy within Kaggle

The competition can be found [here](https://www.kaggle.com/competitions/digit-recognizer/overview).

Inspired by the [MNIST pytorch example](https://github.com/pytorch/examples/blob/main/mnist/main.py).


Import the necessary libraries

In [20]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import pandas as pd

Define the net class

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

Define the training function

In [6]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break

Define the testing function 

In [7]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Define the main loop

In [33]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=21, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--no-mps', action='store_true', default=False,
                        help='disables macOS GPU training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=True,
                        help='For Saving the current Model')
    args, unknown = parser.parse_known_args()
    print(unknown)
    print(args)
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_mps = not args.no_mps and torch.backends.mps.is_available()
    print("Use CUDA: ", use_cuda)
    print("Use MPS: ", use_mps)

    torch.manual_seed(args.seed)

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


if __name__ == '__main__':
    main()

['-f', '/Users/z/Library/Jupyter/runtime/kernel-038056af-988e-45a7-a519-44131911f7ef.json']
Namespace(batch_size=64, test_batch_size=1000, epochs=21, lr=1.0, gamma=0.7, no_cuda=False, no_mps=False, dry_run=False, seed=1, log_interval=10, save_model=True)
Use CUDA:  False
Use MPS:  True
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300024
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.141236
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.711558
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.534700
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.394732
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.266415
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.233061
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.214440
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.619316
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.206452
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.418914
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.228332
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.176347
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.153546
Train Epoch: 1 [8960/60000 (15%)]	Loss: 

Train Epoch: 2 [42240/60000 (70%)]	Loss: 0.018414
Train Epoch: 2 [42880/60000 (71%)]	Loss: 0.022502
Train Epoch: 2 [43520/60000 (72%)]	Loss: 0.055319
Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.011081
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.060235
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.055324
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.103115
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.065135
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.161641
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.040799
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.015006
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.002909
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.115290
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.033574
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.145007
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.009799
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.004715
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.133640
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.132410
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.108191


Train Epoch: 4 [26240/60000 (44%)]	Loss: 0.012240
Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.175181
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.085074
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.064655
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.016856
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.019245
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.031419
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.027690
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.018056
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.019444
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.129972
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.023831
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.000654
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.012946
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.157688
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.116089
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.005845
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.032749
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.006304
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.087044


Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.050458
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.038055
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.060283
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.032902
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.000821
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.065801
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.007074
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.022068
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.049230
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.000486
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.002132
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.038931
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.045500
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.063166
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.002879
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.101686
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.036456
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.011514
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.010502
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.009989


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.037529
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.000374
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.002522
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.001050

Test set: Average loss: 0.0286, Accuracy: 9905/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.006088
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.014722
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.008954
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.005901
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.011460
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.004641
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.000648
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.015815
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.027919
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.102231
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.049844
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.217968
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.011820
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.004765
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.027168
Train Epoch: 8 [9

...

Train Epoch: 14 [51840/60000 (86%)]	Loss: 0.007037
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.000640
Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.003107
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.101709
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.011075
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.000875
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.028875
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.014346
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.001355
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.019705
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.000053
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.001943
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.001069

Test set: Average loss: 0.0273, Accuracy: 9917/10000 (99%)

Train Epoch: 15 [0/60000 (0%)]	Loss: 0.017729
Train Epoch: 15 [640/60000 (1%)]	Loss: 0.000518
Train Epoch: 15 [1280/60000 (2%)]	Loss: 0.003331
Train Epoch: 15 [1920/60000 (3%)]	Loss: 0.042440
Train Epoch: 15 [2560/60000 (4%)]	Loss: 0.008890
Train Epoch: 15 [3200/60000 (5%)]	L

Train Epoch: 16 [34560/60000 (58%)]	Loss: 0.001501
Train Epoch: 16 [35200/60000 (59%)]	Loss: 0.057662
Train Epoch: 16 [35840/60000 (60%)]	Loss: 0.004832
Train Epoch: 16 [36480/60000 (61%)]	Loss: 0.006017
Train Epoch: 16 [37120/60000 (62%)]	Loss: 0.001869
Train Epoch: 16 [37760/60000 (63%)]	Loss: 0.016843
Train Epoch: 16 [38400/60000 (64%)]	Loss: 0.015267
Train Epoch: 16 [39040/60000 (65%)]	Loss: 0.000547
Train Epoch: 16 [39680/60000 (66%)]	Loss: 0.026091
Train Epoch: 16 [40320/60000 (67%)]	Loss: 0.025988
Train Epoch: 16 [40960/60000 (68%)]	Loss: 0.017511
Train Epoch: 16 [41600/60000 (69%)]	Loss: 0.028171
Train Epoch: 16 [42240/60000 (70%)]	Loss: 0.004459
Train Epoch: 16 [42880/60000 (71%)]	Loss: 0.001408
Train Epoch: 16 [43520/60000 (72%)]	Loss: 0.029972
Train Epoch: 16 [44160/60000 (74%)]	Loss: 0.000730
Train Epoch: 16 [44800/60000 (75%)]	Loss: 0.146748
Train Epoch: 16 [45440/60000 (76%)]	Loss: 0.042747
Train Epoch: 16 [46080/60000 (77%)]	Loss: 0.039733
Train Epoch: 16 [46720/60000 (7

Train Epoch: 18 [16640/60000 (28%)]	Loss: 0.046280
Train Epoch: 18 [17280/60000 (29%)]	Loss: 0.000270
Train Epoch: 18 [17920/60000 (30%)]	Loss: 0.023550
Train Epoch: 18 [18560/60000 (31%)]	Loss: 0.038504
Train Epoch: 18 [19200/60000 (32%)]	Loss: 0.018765
Train Epoch: 18 [19840/60000 (33%)]	Loss: 0.030404
Train Epoch: 18 [20480/60000 (34%)]	Loss: 0.020509
Train Epoch: 18 [21120/60000 (35%)]	Loss: 0.021807
Train Epoch: 18 [21760/60000 (36%)]	Loss: 0.000431
Train Epoch: 18 [22400/60000 (37%)]	Loss: 0.018346
Train Epoch: 18 [23040/60000 (38%)]	Loss: 0.009128
Train Epoch: 18 [23680/60000 (39%)]	Loss: 0.015392
Train Epoch: 18 [24320/60000 (41%)]	Loss: 0.000474
Train Epoch: 18 [24960/60000 (42%)]	Loss: 0.002255
Train Epoch: 18 [25600/60000 (43%)]	Loss: 0.012072
Train Epoch: 18 [26240/60000 (44%)]	Loss: 0.032395
Train Epoch: 18 [26880/60000 (45%)]	Loss: 0.022430
Train Epoch: 18 [27520/60000 (46%)]	Loss: 0.018782
Train Epoch: 18 [28160/60000 (47%)]	Loss: 0.084977
Train Epoch: 18 [28800/60000 (4


Test set: Average loss: 0.0273, Accuracy: 9915/10000 (99%)

Train Epoch: 20 [0/60000 (0%)]	Loss: 0.004045
Train Epoch: 20 [640/60000 (1%)]	Loss: 0.000351
Train Epoch: 20 [1280/60000 (2%)]	Loss: 0.034647
Train Epoch: 20 [1920/60000 (3%)]	Loss: 0.016903
Train Epoch: 20 [2560/60000 (4%)]	Loss: 0.002700
Train Epoch: 20 [3200/60000 (5%)]	Loss: 0.002673
Train Epoch: 20 [3840/60000 (6%)]	Loss: 0.000163
Train Epoch: 20 [4480/60000 (7%)]	Loss: 0.005889
Train Epoch: 20 [5120/60000 (9%)]	Loss: 0.006880
Train Epoch: 20 [5760/60000 (10%)]	Loss: 0.100654
Train Epoch: 20 [6400/60000 (11%)]	Loss: 0.071764
Train Epoch: 20 [7040/60000 (12%)]	Loss: 0.123016
Train Epoch: 20 [7680/60000 (13%)]	Loss: 0.009570
Train Epoch: 20 [8320/60000 (14%)]	Loss: 0.000767
Train Epoch: 20 [8960/60000 (15%)]	Loss: 0.061799
Train Epoch: 20 [9600/60000 (16%)]	Loss: 0.004995
Train Epoch: 20 [10240/60000 (17%)]	Loss: 0.026395
Train Epoch: 20 [10880/60000 (18%)]	Loss: 0.001582
Train Epoch: 20 [11520/60000 (19%)]	Loss: 0.008189

Train Epoch: 21 [43520/60000 (72%)]	Loss: 0.000964
Train Epoch: 21 [44160/60000 (74%)]	Loss: 0.003992
Train Epoch: 21 [44800/60000 (75%)]	Loss: 0.011117
Train Epoch: 21 [45440/60000 (76%)]	Loss: 0.018110
Train Epoch: 21 [46080/60000 (77%)]	Loss: 0.005889
Train Epoch: 21 [46720/60000 (78%)]	Loss: 0.040459
Train Epoch: 21 [47360/60000 (79%)]	Loss: 0.028917
Train Epoch: 21 [48000/60000 (80%)]	Loss: 0.053662
Train Epoch: 21 [48640/60000 (81%)]	Loss: 0.041028
Train Epoch: 21 [49280/60000 (82%)]	Loss: 0.016980
Train Epoch: 21 [49920/60000 (83%)]	Loss: 0.018327
Train Epoch: 21 [50560/60000 (84%)]	Loss: 0.012332
Train Epoch: 21 [51200/60000 (85%)]	Loss: 0.059867
Train Epoch: 21 [51840/60000 (86%)]	Loss: 0.001561
Train Epoch: 21 [52480/60000 (87%)]	Loss: 0.000512
Train Epoch: 21 [53120/60000 (88%)]	Loss: 0.006321
Train Epoch: 21 [53760/60000 (90%)]	Loss: 0.095710
Train Epoch: 21 [54400/60000 (91%)]	Loss: 0.015792
Train Epoch: 21 [55040/60000 (92%)]	Loss: 0.000315
Train Epoch: 21 [55680/60000 (9

Use the generated conv net to predict the Kaggle csv test

In [34]:
model = Net()
model.load_state_dict(torch.load('mnist_cnn.pt'))
model.eval()  # set to evaluation mode

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [35]:
data = pd.read_csv('../data/test.csv')
print("Test data length: ", len(data))
images = torch.tensor(data.values, dtype=torch.float32)
images = images.view(-1, 1, 28, 28)

# Predict using loaded model
with torch.no_grad():
    predictions = model(images)
    _, predicted_digits = torch.max(predictions, 1)  # get the class (digit) with highest probability

print(len(predicted_digits))
print(predicted_digits.numpy())
predicted_df = pd.DataFrame(predicted_digits.numpy(), columns=['Label'])
predicted_df.index += 1 
#predicted_df['ImageId'] = predicted_df.index
#print(predicted_df)
predicted_df.to_csv('predicted_digits.csv', index_label='ImageId')

Test data length:  28000
28000
[2 0 9 ... 3 9 2]
